## 1. Introduction/Business Problem

I have a metal band. One of our goals is to improve the reach of our music. In adiition, we want to improve our live shows. One way to work on both goals is to find a set of local venues were we can present our show in a contious fashion. Using the data from the Foursquare database we can identify the density of such venues across different cities. For this study we will compare the data from Manhattan in New York with the data from Toronto named neighborhoods in the city of Toronto. With this comparison we can plan to relocate our band a schedule local shows.

## 2. Data

The Foursquare location data contains information about the venues across different cities. With some work the information about the positions (latitude and longitude) of the neighborhoods of interest can be retrieved from the web. Then this positions can be used to retrieve venues from Foursquare. Venues information include typo of venue. Filtering through music related venues, such as clubs and concert halls will give an idea of which neighborhood may be better suited for a local band.

As usual, first we will import all the libraries we need

In [1]:
import pandas as pd
import numpy as np 
import requests
from bs4 import BeautifulSoup

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Now we will read the data from the neighboorhoods in the city of Toronto which include 'Toronto' in their names.
For this we will scrap a wikipedia page with the postal codes of Canada.

In [2]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969).'
page=requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
table_contents=[]
table=soup.find('table')

In [3]:
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


Now we retrieve the map relating Spatial Coordinates to Postal Codes from our own csv file

In [6]:
df_geodata = pd.read_csv('Geospatial_Coordinates.csv')
df_1 = df.rename(columns={'PostalCode': 'Postal Code'})             
df_complete = pd.merge(df_1, df_geodata, on="Postal Code")

In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_complete['Borough'].unique()),
        df_complete.shape[0]
    )
)
toronto_data = df_complete[df_complete['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

The dataframe has 15 boroughs and 103 neighborhoods.


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Now we will locate the Analogue information for the Manhattan area in New York

In [9]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

zsh:1: command not found: wget
Data downloaded!


In [11]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [12]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Now we have both the Manhatan Data and the Toronto Data. We can now Proceed with the extraction of the Venues on these locations

# Methodology Section 

In order to recover data about the venues we introduce Foursquare

# Foursquare Credentials

In [15]:
CLIENT_ID = 'QXEEFZOJ4OPYPJH21D3B1TDJLIAZMJLSR1HNB5YTEA00RF1K' # your Foursquare ID
CLIENT_SECRET = 'WMRE0HTWPNKRYGE5IZKG50VSMX1B2SSONZIPGKSFB1R4QA2V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Function to retrieve Venues Data

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Finally Retrieve Toronto Venues Data

In [17]:
# type your answer here
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

Retrieve Manhattan Venues Data

In [18]:
# type your answer here
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Note that we can retrive information about the Category of the venues

In [19]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [20]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


# Exploratory Data Analysis

In [21]:
n_toronto = len(pd.unique(toronto_venues['Venue Category']))
n_manhattan = len(pd.unique(manhattan_venues['Venue Category']))

print("No.of.unique values :", n_toronto)
print("No.of.unique values :", n_manhattan)


No.of.unique values : 231
No.of.unique values : 335


Now we can start comparing the number of venues directly related to music between the cities.
A naive first approach consists on looking for Venue Categories including the string music on them.

In [25]:
music_venues_toronto = toronto_venues[toronto_venues['Venue Category'].str.contains("Music")].reset_index(drop=True)
music_venues_manhattan = manhattan_venues[manhattan_venues['Venue Category'].str.contains("Music")].reset_index(drop=True)

print('Toronto Musical Venues are ', len(music_venues_toronto))
print('Manhattan Musical Venues are ', len(music_venues_manhattan))

Toronto Musical Venues are  5
Manhattan Musical Venues are  6


In [26]:
music_venues_toronto.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Garden District, Ryerson",43.657162,-79.378937,Jazz Bistro,43.655678,-79.379276,Music Venue
1,"Dufferin, Dovercourt Village",43.669005,-79.442259,Rehearsal Factory,43.668877,-79.443603,Music Venue
2,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Redpath Stage,43.638764,-79.383317,Music Venue
3,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,WestJet Stage,43.638022,-79.383536,Music Venue
4,"High Park, The Junction South",43.661608,-79.464763,Junction City Music Hall,43.665334,-79.466253,Music Venue


In [27]:
music_venues_manhattan.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown,40.715618,-73.994279,Sofar HQ,40.713523,-73.996289,Music Venue
1,Central Harlem,40.815976,-73.943211,Shrine World Music Venue,40.814403,-73.944242,Music Venue
2,Clinton,40.759101,-73.996119,DiMenna Center for Classical Music,40.756323,-73.997192,Music School
3,Soho,40.722184,-74.000657,Rudy's Music Soho,40.722380,-74.001128,Music Store
4,Noho,40.723259,-73.988434,The Slipper Room,40.721425,-73.988597,Music Venue


As expected, the naive approach showed limited results for both city neighborhoods. Another couple of key words could be the genre of the music my band plays: rock and metal.

In [28]:
rock_venues_toronto = toronto_venues[toronto_venues['Venue Category'].str.contains("Rock")].reset_index(drop=True)
rock_venues_manhattan = manhattan_venues[manhattan_venues['Venue Category'].str.contains("Rock")].reset_index(drop=True)

metal_venues_toronto = toronto_venues[toronto_venues['Venue Category'].str.contains("Metal")].reset_index(drop=True)
metal_venues_manhattan = manhattan_venues[manhattan_venues['Venue Category'].str.contains("Metal")].reset_index(drop=True)

print('Toronto Rock Venues are ', len(rock_venues_toronto))
print('Manhattan Rock Venues are ', len(rock_venues_manhattan))

print('Toronto Metal Venues are ', len(metal_venues_toronto))
print('Manhattan Metal Venues are ', len(metal_venues_manhattan))


Toronto Rock Venues are  0
Manhattan Rock Venues are  6
Toronto Metal Venues are  0
Manhattan Metal Venues are  0


In [29]:
rock_venues_manhattan.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Clinton,40.759101,-73.996119,MPHC Climbing Gym,40.759657,-73.994088,Rock Climbing Spot
1,Lower East Side,40.717807,-73.980890,Parkside Lounge,40.721088,-73.983174,Rock Club
2,Little Italy,40.719324,-73.997305,Bowery Ballroom,40.720328,-73.993335,Rock Club
3,Noho,40.723259,-73.988434,Rockwood Music Hall,40.722196,-73.988841,Rock Club
4,Noho,40.723259,-73.988434,"Rockwood Music Hall, Stage 2",40.722038,-73.988634,Rock Club


**Note** that we have been getting a pair of "false positive" in our searches. Music Schools, Music Stores, and Rock Climbing Spots, may work as venues for a metal band under very particular circumstances. But in no way one could expect to rely on those venues for continuous presentations.

At this point, Manhattan has shown roughly ten different possible venues. Whereas Toronto has around five. But we can keep digging. Some great locations for a concert may be found with excatly that keyword: Concert.

In [30]:
concert_venues_toronto = toronto_venues[toronto_venues['Venue Category'].str.contains("Concert")].reset_index(drop=True)
concert_venues_manhattan = manhattan_venues[manhattan_venues['Venue Category'].str.contains("Concert")].reset_index(drop=True)

general_venues_toronto = toronto_venues[toronto_venues['Venue Category'].str.contains("General Entertainment")].reset_index(drop=True)
general_venues_manhattan = manhattan_venues[manhattan_venues['Venue Category'].str.contains("General Entertainment")].reset_index(drop=True)

print('Toronto Concert Venues are ', len(rock_venues_toronto))
print('Manhattan Concert Venues are ', len(rock_venues_manhattan))

print('Toronto General Entertainment Venues are ', len(metal_venues_toronto))
print('Manhattan General Entertainment Venues are ', len(metal_venues_manhattan))


Toronto Concert Venues are  0
Manhattan Concert Venues are  6
Toronto General Entertainment Venues are  0
Manhattan General Entertainment Venues are  0


In [31]:
concert_venues_manhattan.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lincoln Square,40.773529,-73.985338,New York Philharmonic,40.772526,-73.983139,Concert Hall
1,Lincoln Square,40.773529,-73.985338,David Geffen Hall,40.772305,-73.983156,Concert Hall
2,Lincoln Square,40.773529,-73.985338,Alice Tully Hall,40.773359,-73.982373,Concert Hall
3,Lincoln Square,40.773529,-73.985338,Merkin Concert Hall,40.775179,-73.983130,Concert Hall
4,Midtown,40.754691,-73.981669,The Town Hall,40.756058,-73.984595,Concert Hall


Again, Manhattan shows more options. However the results are mostly larger venues. Continuing with this train of thought, festivals and stadiums are great arenas for consolidated bands. If we want to succeed eventually we have to look for those venues as the opportunities arise.

In [32]:
stadium_venues_toronto = toronto_venues[toronto_venues['Venue Category'].str.contains("Stadium")].reset_index(drop=True)
stadium_venues_manhattan = manhattan_venues[manhattan_venues['Venue Category'].str.contains("Stadium")].reset_index(drop=True)

festival_venues_toronto = toronto_venues[toronto_venues['Venue Category'].str.contains("Festival")].reset_index(drop=True)
festival_venues_manhattan = manhattan_venues[manhattan_venues['Venue Category'].str.contains("Festival")].reset_index(drop=True)

print('Toronto Stadium Venues are ', len(stadium_venues_toronto))
print('Manhattan Stadium Venues are ', len(stadium_venues_manhattan))

print('Toronto Festival Venues are ', len(festival_venues_toronto))
print('Manhattan Festival Venues are ', len(festival_venues_manhattan))


Toronto Stadium Venues are  7
Manhattan Stadium Venues are  2
Toronto Festival Venues are  0
Manhattan Festival Venues are  0


In [33]:
stadium_venues_toronto.head(7)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,43.644771,-79.373306,Scotiabank Arena,43.643446,-79.379040,Basketball Stadium
1,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Scotiabank Arena,43.643446,-79.379040,Basketball Stadium
2,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Rogers Centre,43.641753,-79.387150,Baseball Stadium
3,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Toronto Blue Jays Box Office,43.642416,-79.385862,Baseball Stadium
4,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,Scotiabank Arena,43.643446,-79.379040,Basketball Stadium
5,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,Lamport Stadium,43.638778,-79.423534,Stadium
6,Enclave of M5E,43.646435,-79.374846,Scotiabank Arena,43.643446,-79.379040,Basketball Stadium


In [34]:
stadium_venues_manhattan.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,TCR The Club of Riverdale,40.878628,-73.914568,Tennis Stadium
1,Murray Hill,40.748303,-73.978332,Vanderbilt Tennis Club,40.752252,-73.977477,Tennis Stadium


It is important to keep in mind that festivals may occur in parks.

In [35]:
park_venues_toronto = toronto_venues[toronto_venues['Venue Category'].str.contains("Park")].reset_index(drop=True)
park_venues_manhattan = manhattan_venues[manhattan_venues['Venue Category'].str.contains("Park")].reset_index(drop=True)

print('Toronto Park Venues are ', len(park_venues_toronto))
print('Manhattan Park Venues are ', len(park_venues_manhattan))


Toronto Park Venues are  37
Manhattan Park Venues are  73


Now that we have an idea of the number of musical venues creating a visualization of them could help to forsee logistic details of touring with a band in these places.

**Toronto**

In [38]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


# create map of Toronto using latitude and longitude values
map_toronto_ = folium.Map(location=[latitude, longitude], zoom_start=12)

# add Blue markers to map for music venues
for lat, lng, label in zip(music_venues_toronto['Venue Latitude'], music_venues_toronto['Venue Longitude'], music_venues_toronto['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_)  

    
# add Red markers to map for stadiums
for lat, lng, label in zip(stadium_venues_toronto['Venue Latitude'], stadium_venues_toronto['Venue Longitude'], stadium_venues_toronto['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_)  

map_toronto_


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


**Manhattan**

In [39]:
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))


# create map of Toronto using latitude and longitude values
map_manhattan_ = folium.Map(location=[latitude, longitude], zoom_start=12)

# add Blue markers to map for music venues
for lat, lng, label in zip(music_venues_manhattan['Venue Latitude'], music_venues_manhattan['Venue Longitude'], music_venues_manhattan['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_)  

    
# add Red markers to map for rock clubs
for lat, lng, label in zip(rock_venues_manhattan['Venue Latitude'], rock_venues_manhattan['Venue Longitude'], rock_venues_manhattan['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_)  

# add green markers to map for stadiums
for lat, lng, label in zip(stadium_venues_manhattan['Venue Latitude'], stadium_venues_manhattan['Venue Longitude'], stadium_venues_manhattan['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_)  

    
map_manhattan_


The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


# Results Section

It can be seen from the exploratory analysis that Manhattan has both more venues suited for a metal band to perform. It is also evident that the venues are closer to each other in Manhattan providing an easier opportunity for transporting the equipment necessary to perform a show.

# Discussion Section

It is important to note that the present analysis is very limited in scope. The two neighborhoods compared both have stadiums and concert Halls. But those venues are more suited to large events. A forcoming band that needs to develop experience prefers a continuous set of shows to improve their performance. And the availability of numerous rock clubs in Manhattan is better suited for this proposes.

# Conclusion Section

This could be a useful preliminary result in a more comprehensive study. The limited scope (two neighborhoods in to major cities) implies the possibility to extend the exploration. In addition to adding more cities, different kinds of venues may be used for concerts in different parts of the world. An investigation of the local culture in different places is important to add context to the data. In this direction, maybe different parts of the world have not updated their venue locations into the Foursquare platform as exhaustively as others.

In conclusion, a comparison between the venues appropiate for an upcoming metal band (Tuner_mx) was perforemed. The data from the neighborhoods of Manhattan in NY and Toronto in Canada ware scrapped from the web. The exploration of the venues was done using the Foursquare database. The results show a higher concentration of Rock Music Clubs in Manhattan. 